In [387]:
# Importing the packages we will need
from __future__ import print_function
from __future__ import division
from IPython.core.display import HTML
from IPython.display import display, clear_output
from ipywidgets import Button, Layout, GridBox, ButtonStyle, HBox, VBox, Label, Dropdown
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML
from matplotlib import pyplot as plt

import ipywidgets as ipw
%matplotlib inline
import pandas as pd
import math
import numpy as np
import xlrd
import xlsxwriter
import os.path
import glob
import warnings
import platform
import time

import scipy
from scipy.io import loadmat
from scipy.sparse import linalg
import os
import sys

HTML("""
<style>
#notebook-container{
    box-shadow: none !important;
}

.body {
    max-width: 960px !important;
}

div.alert alert-block alert-info{
    max-width: 960px !important;
}

.container {
    max-width: 960px !important;
}

.notebook_app {
    background: #fff !important;
}

.navbar-default {
    background: none;
    border: none;
}

.navbar-default .navbar-nav > li > a:hover, #kernel_indicator:hover {
    border-bottom: 2px solid #fff;
    color: rgba(255, 255, 255, 1);
}

div.input_area {
    border: none;
    border-radius: 0;
    background: #f7f7f7;
    line-height: 1.5em;
    margin: 0.5em 0;
    padding: 0;
    max-width: 960px !important;
}

div.cell {
    transition: all 0.25s;
    border: none;
    position: relative;
    top: 0;
    max-width: 960px !important;
}

div.cell.selected, div.cell.selected.jupyter-soft-selected {
    border: none;
    background: transparent;
    box-shadow: 0 6px 18px #aaa;
    z-index: 10;
    top: -10px;
    max-width: 960px !important;
}


div#pager {
    opacity: 0.85;
    z-index: 9999;
    max-width: 960px !important;
}

.navbar-default .navbar-nav > .open > a, .navbar-default .navbar-nav > .open > a:hover, .navbar-default .navbar-nav > .open > a:focus {
    color: #fff;
    background-color: transparent;
    border-bottom: 2px solid #fff;
}

.dropdown-menu {
    z-index: 999999 !important;
    opacity: 0.95;
    max-width: 960px !important;
}

.dropdown-menu > li > a {
    color: #fff;
}

.dropdown-menu > .disabled > a, .dropdown-menu > .disabled > a:hover, .dropdown-menu > .disabled > a:focus {
    color: rgba(255, 255, 255, 0.25);
}

.navbar-nav > li > .dropdown-menu {
    border: none;
    box-shadow: none;
}

div.output_wrapper {
    background: #fff;
}

div.cell.selected .div.output_scroll {
    box-shadow: none;
    max-width: 960px !important;
}

div.output_wrapper {
    margin: 0 0 1em;
    transition: all 0.25s;
    max-width: 960px !important;
}

div.cell.selected .output_wrapper {
    margin: 0;
}

.dataframe {
    background: #fff;
    box-shadow: 0px 1px 2px #bbb;
}

.dataframe thead th, .dataframe tbody td {
    text-align: right;
    padding: 1em;
}

.output, div.output_scroll {
    box-shadow: none;
}

.rendered_html pre code {
    background: #f4f4f4;
    border: 1px solid #ddd;
    border-left: 3px solid #2a7bbd;
    color: #444;
    page-break-inside: avoid;
    font-family: monospace;
    font-size: 15px;
    line-height: 1.6;
    margin-bottom: 1.6em;
    max-width: 600px;
    overflow: auto;
    padding: 1em 1.5em;
    display: block;
    word-wrap: break-word;
}

h1, .h1 {
    font-size: 33px;
    font-family: "Trebuchet MS";
    font-size: 2.5em !important;
    color: #2a7bbd;
}
</style>
""")

In [396]:
def ice_flow_model_02(accu, accu_slope, ELA_0, t_max, h_start): 
    A_0 = 1e-24
    rho_i = 910
    g = 9.81
    n = 3.0
    C = 2.0* A_0 * (rho_i*g)**n/(n+1)

    try:
        x=np.linspace(0, 10000, num=50)
        bed=-10.0-x/100
        h = h_start
        #i_0 = np.where( h< 0)
        #h[i_0]=0
        h=h.astype('float128')
        s=bed+h
        #accu_array=(accu-x*accu_slope + ELA_0*s)/(365*24*60*60) 
    except:
        print("Overflow Error 0")

    # Data for plotting
    t = np.linspace(0, t_max*(365*24*60*60), num=round(t_max/1000)*5000*len(x))
    L=0*t

    delta_t=t[2]-t[1]
    delta_x=x[2]-x[1]
    k=0

    #fig, axs = plt.subplots(ncols=3, nrows=2)
    #fig.set_size_inches(18.5, 5.5, forward=True)
    plt.rcParams.update({'font.size': 22})
    fig = plt.figure(constrained_layout=False)
    fig.set_size_inches(18.5, 7.5, forward=True)
    gs = fig.add_gridspec(2, 3)
    gs.update(wspace = 0.5, hspace = 1.0)


    ax1 = fig.add_subplot(gs[:, :-1])
    ax1.set_title('gs[:, :-1]')
    ax2 = fig.add_subplot(gs[0, 2])
    ax2.set_title('gs[1, 2]')
    ax3 = fig.add_subplot(gs[1, 2])
    ax3.set_title('gs[2, 2]')
    
    
    for i in t/(365*24*60*60):
        s=bed+h
        
        if any(h>1.0):

            try:
                ds_dx_p=(s[2:]-s[1:-1])/delta_x   # 1.5 to end-0.5
                ds_dx_m=(s[1:-1]-s[:-2])/delta_x  # 0.5 to end-1.5
                h_np1=np.power(h,(n+1))
                q_p=-(h_np1[2:]   + h_np1[1:-1])/2.0 * np.power((np.abs(ds_dx_p)),(n-1.0))*ds_dx_p
            
                q_m=-(h_np1[1:-1] + h_np1[:-2] )/2.0 * np.power((np.abs(ds_dx_m)),(n-1.0))*ds_dx_m 
                dq_dx=q_p-q_m  # 1.0 to end-1.0

                q=0*h
                q[1:-1] = C*(q_p+q_m)/2.0
                u=q/(np.abs(h)+1e-6)
                accu_array=(accu-x*accu_slope + ELA_0*s)/(365*24*60*60)   
                h[1:-1]=h[1:-1] - C*delta_t/delta_x*dq_dx + accu_array[1:-1]*delta_t
            
                h[0] = h[1] + bed[1] - bed[0]
                h[-1] = h[-2] + bed[-2] - bed[-1]
                i_0 = np.where( h < 0)
                h=np.where(h <= 0, 0, h)

                u=np.where(h < 0, 0, u)
                u[:-1]=np.where(h[1:] <= 0, 0, u[:-1])
            except OverflowError:
                print("Overflow Error 1")
        else:
            try:
                accu_array=(accu-x*accu_slope + ELA_0*s)/(365*24*60*60)   
                h=h + accu_array*delta_t
                i_0 = np.where( h < 0)
                h=np.where(h <= 0, 0, h)
            except:
                print("Overflow Error 2")
            
        i_0=np.nonzero(h[:-1]) 
        if len(np.nonzero(h[:-1])[0])==0:
            L[k]=0
        else:
            L[k]=x[np.argmax(i_0)]
            h[np.argmax(i_0)+1:]=0 # just in case, setting to zero what should be zero

        if k % (500*len(x))==0:

            plt.clf()  

            ax1 = fig.add_subplot(gs[0, 2])
            ax1.plot(x/1000, accu_array*(365*24*60*60))
            ax1.set(xlabel='x [km]', ylabel='accu [m/yr]',title='Surface mass balance',
                    xlim=(0, 10))
            ax1.grid()
            
            ax2 = fig.add_subplot(gs[:, :-1])
            ax2.fill_between(x/1000, bed, 0*bed-150, facecolor="peru",edgecolor="peru" )
            ax2.plot(x/1000, bed+h,linewidth=2,color="black")

            ax2.set(xlabel='x [km]', ylabel='z [m]',title='t=%i years' %round(i,0),
                    xlim=(0, 10), ylim=(-120,bed[0]+h[0]+10))
            ax2.grid()

            ax3 = fig.add_subplot(gs[1, 2])
            ax3.plot(t[:k-1]/(365*24*60*60), L[:k-1]/1000)
            ax3.set(xlabel='t [yrs]', ylabel='L [km]', xlim=(0, t_max), ylim=(0, 10),
                   title='Length=%3.3f km' %(round(L[k])/1000))  

            display(plt.gcf())
            
            clear_output(wait=True)

        k=k+1

    plt.clf()  
    ax1 = fig.add_subplot(gs[0, 2])
    ax1.plot(x/1000, accu_array*(365*24*60*60))
    ax1.set(xlabel='x [km]', ylabel='$\dot a$ [m/yr]', title='Surface mass balance',
                    xlim=(0, 10))
    ax1.grid()

    ax2 = fig.add_subplot(gs[:, :-1])
    ax2.fill_between(x/1000, bed, 0*bed-150, facecolor="peru",edgecolor="peru" )
    ax2.plot(x/1000, bed+h,color="black")
    ax2.set(xlabel='x [km]', ylabel='z [m]',title='t=%i years (final)' %round(i,0),
                    xlim=(0, 10), ylim=(-120,bed[0]+h[0]+10))
    ax2.grid()

    ax3 = fig.add_subplot(gs[1, 2])
    ax3.plot(t[:k-1]/(365*24*60*60), L[:k-1]/1000)
    ax3.set(xlabel='t [yrs]', ylabel='L [km]', xlim=(0, t_max), ylim=(0, 10),
            title='Length=%3.3f km' %(round(L[k-1])/1000))

    display(plt.gcf())
    clear_output(wait=True)
                      
    return h

In [393]:
def getvalue(x): 
    return x

# Make user interface

style = {'description_width': '200px'}
w3 = interactive(getvalue, x=ipw.BoundedFloatText(
    value=-1.0,
    min=-30.0,
    max=30.0,
    step=0.01,
    description='$T_{surf}$ in $^{o}$C:',
    style=style,
    disabled=False
))
# display(w1)

w4 = interactive(getvalue, x=ipw.BoundedFloatText(
    value=0.001,
    min=0,
    max=0.01,
    step=0.0001,
    description=r'\(\beta\) in m yr\(^{-1}\) m\(^{-1}\):',
    style=style,
    disabled=False
))
w5 = interactive(getvalue, x=ipw.BoundedFloatText(
    value=0.0,
    min=0,
    max=10.0,
    step=0.001,
    description='\(\gamma\) in m year$^{-1}$:',
    style=style,
    disabled=False
))
w6 = interactive(getvalue, x=ipw.BoundedFloatText(
    value=1000,
    min=0,
    max=10000.0,
    step=1,
    description='$t_{max}$ in years:',
    style=style,
    disabled=False
))
# display(w1)

# KE3001 Mass balance practical
<div class="body">
<b>Welcome!</b> In this practical you will run a real ice sheet model that solves for ice sheet mass balance and velocity. In the model below, the surface mass balance (accumulation and ablation) is given by 
    $$\dot a=\alpha \times T_{surf} - \beta \times x + \gamma\times s.$$
    
    
<ul>
    <li>$\alpha=-1$ m year$^{-1}$ $(^oC)^{-1}$: Proportionality factor 
    <li>$\beta=0.001$ year$^{-1}$: Latitudinal accumulation gradient
    <li>$\gamma$ in year$^{-1}$: Elevational accumulation gradient
    <li>$t_{max}$ in years: Maximum number of model years
    <li>$T_{surf}$ in $^o$C: average surface temperature at $x=0$, $s=0$
    <li>$x$ in m: distance from the glacier start
    <li>$s(x)$ in m: is the surface height of the glacier (changes with $x$)
</ul>
    
    
You can change $T_{surf}$, $\beta$, $\gamma$ and the number of years the model is run for. The final state of the last run becomes the initial state of the next run. If you want to start from zero ice thickness instead, click reset first.
</div>



In [390]:
left_box  = ipw.VBox([w3, w4])
right_box = ipw.VBox([w5, w6])
ipw.HBox([left_box,right_box])

#left_box  = ipw.VBox([w3])
#center_box  = ipw.VBox([w4])
#right_box  = ipw.VBox([w5])
#ipw.HBox([left_box,center_box,right_box])

In [391]:
# Loading the input parameters defined by the user
button1 = ipw.Button(description="Run the model!")
output1 = ipw.Output()
button2 = ipw.Button(description="Reset")
output2 = ipw.Output()

display(button1, output1)
display(button2, output2)
global iter, h_initial
x=np.linspace(0, 1000, num=50)
h_initial=0 - 0*np.power(x/100,2)

iter=0
def on_button_clicked1(b):
    with output1:
        clear_output(wait=True)
        accu=-w3.result
        accu_slope=w4.result
        ELA_0=w5.result
        t_max=w6.result
        
        global iter, h_final, h_initial
        iter=iter+1 
        h_final=ice_flow_model_02(accu, accu_slope, ELA_0, t_max, h_initial)
        h_initial=h_final
button1.on_click(on_button_clicked1)

def on_button_clicked2(b):
    global h_initial
    h_initial=0 - 0*np.power(x/100,2)
        
button2.on_click(on_button_clicked2)

Button(description='Run the model!', style=ButtonStyle())

Output()

Button(description='Reset', style=ButtonStyle())

Output()

## What you will learn
<div class="body">
<ul>
<li> Principles of ice sheet and glacier mass balance
<li> Reversible and irreversible processes, hysteresis
<li> Creating plots in Excel, incl. fitting data with a trendline.
</ul>
</div>

## Instructions part 1: Ice sheet mass balance
<div class="body"><div class="alert alert-block alert-info"> 
Run the model into steady state for at least 5 different values of $T_{surf}$ between $0^c$C and $-4^o$C with $\gamma=0$. Note the values of $T_{surf}$ and the final length of the glacier for each of your runs and answer the following questions.
</div>
<div class="body">
<ol>
<li> Explain the shape of the surface mass balance plot.
<li> As you change $T_{surf}$, how does the surface mass balance change?
<li> What is a steady state? How long does it typically take for the glacier to reach steady state?
<li> Create a scatter graph with $T_{surf}$ on the $x$-axis and the final glacier length on the $y$ axis. What do you observe? Add a lienar trendline and note the equation of the trendline.
<li> What alternative way is there to determine the relationship between $T_{surf}$ and the final glacier length $L$?
 </ol>
</div></div>

<div class="body">
<h2>Instructions part 2: Mass balance-elevation feedback</h2>
<div class="alert alert-block alert-info"> 
Starting from a steady-state solution for $T_{surf}=-4^o$C, change the value of the elevational accumulation gradient $\gamma$ to $\gamma=0.005$.  Increase the surface temperature from $0^o$C to $0.38^o$C, taking at least 5 steps. Run the glacier into steady state for each value of $T_{surf}$, making sure that the length of the glacier does not change for at least 2000 model years. Note the steady state length and surface temperature for each run and answer the following questions.</div>
<ol>
<li> What does the surface accumulation look like now? Why?
<li> Create a scatter graph with $T_{surf}$ on the $x$-axis and the final glacier length on the $y$ axis. What do you observe? Is there a linear relationship between the surface temperature and the final glacier length?
<li> Once the glacier has disappeared, reduce the surface temperature again, taking the same steps you have taken previously. Note the corresponding glacier lengths for these runs too and add these points to you scatter plot. Does the glacier return to its previous length?

<li> How much do you have to decrease the surface temperature to build a glacier back up?
</ol>
</div>

### 